In [392]:
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [393]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [394]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]


In [395]:
# 30 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,3,1)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
ohlc_df

,time,open,high,low,close
0,2025-03-03 00:00:00,1.03925,1.03954,1.03925,1.03954
1,2025-03-03 00:01:00,1.03954,1.03970,1.03947,1.03970
2,2025-03-03 00:02:00,1.03970,1.03983,1.03940,1.03957
3,2025-03-03 00:03:00,1.03946,1.03978,1.03946,1.03972
4,2025-03-03 00:04:00,1.03972,1.03972,1.03965,1.03965
...,...,...,...,...,...
64666,2025-05-02 22:55:00,1.13053,1.13057,1.13048,1.13049
64667,2025-05-02 22:56:00,1.13049,1.13062,1.13048,1.13053
64668,2025-05-02 22:57:00,1.13054,1.13061,1.13051,1.13061
64669,2025-05-02 22:58:00,1.13061,1.13064,1.13057,1.13059


In [396]:
# hour4 df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
hourly_df.tail(5)

,time,open,high,low,close
265,2025-05-02 04:00:00,1.12823,1.13157,1.12786,1.13044
266,2025-05-02 08:00:00,1.13044,1.13379,1.13025,1.13324
267,2025-05-02 12:00:00,1.13330,1.13551,1.13047,1.13190
268,2025-05-02 16:00:00,1.13190,1.13810,1.13149,1.13199
269,2025-05-02 20:00:00,1.13199,1.13204,1.12926,1.12994


In [397]:
# daily df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# Convert 'time' column to full timestamp
#daily_df['time'] = pd.to_datetime(daily_df['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
#daily_df['time'] = pd.to_datetime(daily_df['time'], unit= 's').dt.strftime('%Y-%m-%d %H:%M:%S')
daily_df

,time,open,high,low,close
0,2025-03-03,1.03925,1.05038,1.03887,1.04878
1,2025-03-04,1.04860,1.06270,1.04712,1.06251
2,2025-03-05,1.06253,1.07968,1.06021,1.07892
3,2025-03-06,1.07866,1.08534,1.07656,1.07838
4,2025-03-07,1.07823,1.08889,1.07789,1.08321
5,2025-03-10,1.08325,1.08747,1.08050,1.08354
6,2025-03-11,1.08330,1.09474,1.08260,1.09193
7,2025-03-12,1.09180,1.09301,1.08757,1.08871
8,2025-03-13,1.08870,1.08973,1.08225,1.08514
9,2025-03-14,1.08503,1.09128,1.08307,1.08802


In [398]:
# Calculate Pivot Point, R1, R2, R3, stopeR4, S1, S2, S3, stopeS4 levels with Fibonacci 
daily_df["pivot"] = ((daily_df["high"] + daily_df["low"] + daily_df["close"]) / 3).shift(1)
daily_df["prange"] = (daily_df["high"] - daily_df["low"]).shift(1)
daily_df["R1"] = daily_df["pivot"] + 0.382 * daily_df["prange"]
daily_df["R2"] = daily_df["pivot"] + 0.618 * daily_df["prange"]
daily_df["R3"] = daily_df["pivot"] + 1.000 * daily_df["prange"]

daily_df["S1"] = daily_df["pivot"] - 0.382 * daily_df["prange"]
daily_df["S2"] = daily_df["pivot"] - 0.618 * daily_df["prange"]
daily_df["S3"] = daily_df["pivot"] - 1.000 * daily_df["prange"]
daily_df[["time","S2","R2"]].tail(10)


,time,S2,R2
35,2025-04-21,1.135896,1.140704
36,2025-04-22,1.136918,1.160822
37,2025-04-23,1.138156,1.154211
38,2025-04-24,1.127271,1.143562
39,2025-04-25,1.131077,1.142003
40,2025-04-28,1.130904,1.140223
41,2025-04-29,1.133295,1.145099
42,2025-04-30,1.135971,1.142349
43,2025-05-01,1.129784,1.139956
44,2025-05-02,1.125253,1.134547


In [399]:
#setting the sma 1 and 14 periods
furst_sma_period = 1
second_sma_period = 14
#ohlc4
hourly_df['ohlc4'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4

# sma 1
hourly_df['sma_1'] = hourly_df['ohlc4'].rolling(furst_sma_period).mean()
hourly_df['prev_sma_1'] = hourly_df['sma_1'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 14
hourly_df['sma_14'] = hourly_df['ohlc4'].rolling(second_sma_period).mean()
hourly_df

,time,open,high,low,close,ohlc4,sma_1,prev_sma_1,sma_14
0,2025-03-03 00:00:00,1.03925,1.04232,1.03907,1.04199,1.040658,1.040658,NaN,NaN
1,2025-03-03 04:00:00,1.04199,1.04234,1.04100,1.04144,1.041692,1.041692,1.040658,NaN
2,2025-03-03 08:00:00,1.04144,1.04290,1.03887,1.04273,1.041485,1.041485,1.041692,NaN
3,2025-03-03 12:00:00,1.04275,1.04764,1.04273,1.04721,1.045082,1.045082,1.041485,NaN
4,2025-03-03 16:00:00,1.04721,1.05038,1.04688,1.04904,1.048377,1.048377,1.045082,NaN
...,...,...,...,...,...,...,...,...,...
265,2025-05-02 04:00:00,1.12823,1.13157,1.12786,1.13044,1.129525,1.129525,1.128623,1.133181
266,2025-05-02 08:00:00,1.13044,1.13379,1.13025,1.13324,1.131930,1.131930,1.129525,1.132712
267,2025-05-02 12:00:00,1.13330,1.13551,1.13047,1.13190,1.132795,1.132795,1.131930,1.132380
268,2025-05-02 16:00:00,1.13190,1.13810,1.13149,1.13199,1.133370,1.133370,1.132795,1.132073


In [400]:
#setting the 11 and 114 sma periods
thurd_sma_period = 1
fourth_sma_period = 14
#ohlc4
daily_df['ohlc4'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
# sma 11
daily_df['sma_11'] = daily_df['ohlc4'].rolling(thurd_sma_period).mean()
daily_df['prev_sma_11'] = daily_df['sma_11'].shift(1) #to find crossovers, previous sma value is necessary using shift()
# sma 114
daily_df['sma_114'] = daily_df['ohlc4'].rolling(fourth_sma_period).mean()
daily_df[["time","sma_11","sma_114","prev_sma_11"]]

,time,sma_11,sma_114,prev_sma_11
0,2025-03-03,1.044320,NaN,NaN
1,2025-03-04,1.055233,NaN,1.044320
2,2025-03-05,1.070335,NaN,1.055233
3,2025-03-06,1.079735,NaN,1.070335
4,2025-03-07,1.082055,NaN,1.079735
5,2025-03-10,1.083690,NaN,1.082055
6,2025-03-11,1.088143,NaN,1.083690
7,2025-03-12,1.090272,NaN,1.088143
8,2025-03-13,1.086455,NaN,1.090272
9,2025-03-14,1.086850,NaN,1.086455


In [401]:
#setting entring and closing time and one trade a day
# Set the start and end times for your desired trading hours
start_time = datetime.strptime('06:00:00', '%H:%M:%S').time()
end_time = datetime.strptime('17:00:00', '%H:%M:%S').time()
end_trading_section = datetime.strptime('23:45:00', '%H:%M:%S').time()

# Filter the trade data for trades within the specified trading hours
#trades_within_hours = daily_df[(daily_df['time'].dt.time >= start_time) & (daily_df['time'].dt.time <= end_time)]
# open one trade a day
first_trade_per_day = daily_df.groupby(daily_df['time'].dt.time).first()
# Close all open positions at 11:45 PM
open_positions = daily_df[daily_df['time'].dt.time == end_trading_section]

# function to trading
daily_df["sell_stope_loss"] = (daily_df["R2"] - daily_df["S1"]) /2 + daily_df["R2"] 
daily_df["buy_stope_loss"] = (daily_df["S2"] - ((daily_df["R1"] - daily_df["S2"]) /2))
daily_df['sell_profit'] = daily_df['low'] <= daily_df["S1"]
daily_df['buy_profit'] = daily_df["high"] >= daily_df["R1"]
daily_df

,time,open,high,low,close,pivot,prange,R1,R2,R3,...,S2,S3,ohlc4,sma_11,prev_sma_11,sma_114,sell_stope_loss,buy_stope_loss,sell_profit,buy_profit
0,2025-03-03,1.03925,1.05038,1.03887,1.04878,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.044320,1.044320,NaN,NaN,NaN,NaN,False,False
1,2025-03-04,1.04860,1.06270,1.04712,1.06251,1.046010,0.01151,1.050407,1.053123,1.057520,...,1.038897,1.034500,1.055233,1.055233,1.044320,NaN,1.058878,1.033142,False,True
2,2025-03-05,1.06253,1.07968,1.06021,1.07892,1.057443,0.01558,1.063395,1.067072,1.073023,...,1.047815,1.041863,1.070335,1.070335,1.055233,NaN,1.074862,1.040025,False,True
3,2025-03-06,1.07866,1.08534,1.07656,1.07838,1.072937,0.01947,1.080374,1.084969,1.092407,...,1.060904,1.053467,1.079735,1.079735,1.070335,NaN,1.094704,1.051169,False,True
4,2025-03-07,1.07823,1.08889,1.07789,1.08321,1.080093,0.00878,1.083447,1.085519,1.088873,...,1.074667,1.071313,1.082055,1.082055,1.079735,NaN,1.089909,1.070277,False,True
5,2025-03-10,1.08325,1.08747,1.08050,1.08354,1.083330,0.01100,1.087532,1.090128,1.094330,...,1.076532,1.072330,1.083690,1.083690,1.082055,NaN,1.095628,1.071032,False,False
6,2025-03-11,1.08330,1.09474,1.08260,1.09193,1.083837,0.00697,1.086499,1.088144,1.090807,...,1.079529,1.076867,1.088143,1.088143,1.083690,NaN,1.091629,1.076044,False,True
7,2025-03-12,1.09180,1.09301,1.08757,1.08871,1.089757,0.01214,1.094394,1.097259,1.101897,...,1.082254,1.077617,1.090272,1.090272,1.088143,NaN,1.103329,1.076184,False,False
8,2025-03-13,1.08870,1.08973,1.08225,1.08514,1.089763,0.00544,1.091841,1.093125,1.095203,...,1.086401,1.084323,1.086455,1.086455,1.090272,NaN,1.095845,1.083681,True,False
9,2025-03-14,1.08503,1.09128,1.08307,1.08802,1.085707,0.00748,1.088564,1.090329,1.093187,...,1.081084,1.078227,1.086850,1.086850,1.086455,NaN,1.094069,1.077344,False,True


In [402]:
ohlc_df = ohlc_df.copy()
ohlc_df['time'] = pd.to_datetime(ohlc_df['time'])
ohlc_df = ohlc_df.sort_values('time')

daily_df = daily_df.copy()
daily_df['time'] = pd.to_datetime(daily_df['time'])
daily_df = daily_df.sort_values('time')
def entring_stoploss_profit(row, ohlc_df) :
    if row["prev_sma_11"] > row['sma_114'] and row["low_x"] <= row["S2"] :
        return "buy"
    elif row['prev_sma_11'] < row['sma_114'] and row["high_x"] >= row["R2"]:
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df = pd.merge_asof(ohlc_df, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)
merged_df[["signal","time"]].dropna()



,signal,time
19447,buy,2025-03-20 12:24:00
19448,buy,2025-03-20 12:25:00
19449,buy,2025-03-20 12:26:00
19450,buy,2025-03-20 12:27:00
19451,buy,2025-03-20 12:28:00
...,...,...
64389,sell,2025-05-02 18:18:00
64390,sell,2025-05-02 18:19:00
64391,sell,2025-05-02 18:20:00
64392,sell,2025-05-02 18:21:00


In [403]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "low_y": "daily_low",
    "high_y": "daily_high",
    "open_y": "daily_open",
    "close_y": "daily_close"
})


In [404]:
print(merged_df.columns.tolist())



['time', 'open', 'high', 'low', 'close', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'pivot', 'prange', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', 'ohlc4', 'sma_11', 'prev_sma_11', 'sma_114', 'sell_stope_loss', 'buy_stope_loss', 'sell_profit', 'buy_profit', 'signal']


In [405]:
# create trade logic
def on_bar(data, trades, orders):
    #daily_df['time'] = pd.to_datetime(daily_df['time']).dt.strftime('%Y-%m-%d %H:%M:%S')
     # 0.25 lots
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    if 'signal' not in data:
        print("Warning: 'signal' column is missing!")
        return
    account_balance = 10000  # your account in USD
    risk_percent = 0.04  # 4%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        stop_loss_pips = data['S2'] - (data["S2"] - ((data["R1"] - data["S2"]) /2)) # example SL
        volume = 25000#risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        stop_loss_pips = ((data["R2"] - data["S1"]) /2 + data["R2"]) - data["R2"]
        volume = 25000#risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and ((data["low"] <= data["S2"] - ((data["R1"] - data["S2"]) /2)) 
                                            or (data["high"] >= data["R1"]) 
                                            or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and ((data["high"] >= (data["R2"] - data["S1"]) /2 + data["R2"]) 
                                            or (data["low"] <= data["S1"]) 
                                            or (data['time'].time() == pd.to_datetime('00:00:00').time())):
            orders.close_trade(trade)

In [414]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades.head(5)

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,25000,2025-03-20 12:24:00,1.08503,2025-03-21 00:00:00,1.08494,0,0,{},-2.25,-1.75,-4.0,-4.0,9996.0
1,closed,EURUSD,buy,25000,2025-03-21 18:53:00,1.08077,2025-03-24 00:00:00,1.08127,0,0,{},12.50,-1.75,10.75,6.75,10006.75
2,closed,EURUSD,sell,25000,2025-03-27 18:31:00,1.08007,2025-03-28 00:00:00,1.08011,0,0,{},-1.00,-1.75,-2.75,4.0,10004.0
3,closed,EURUSD,sell,25000,2025-03-28 18:23:00,1.08378,2025-03-31 00:00:00,1.08225,0,0,{},38.25,-1.75,36.5,40.5,10040.5
4,closed,EURUSD,sell,25000,2025-04-02 17:41:00,1.08282,2025-04-02 17:52:00,1.08523,0,0,{},-60.25,-1.75,-62.0,-21.5,9978.5


In [407]:
mm = bt.plot_balance()
display(mm)

In [408]:
pnl_chart = bt.plot_pnl()
display(pnl_chart)

In [409]:
evaluate_backtest(bt.trades)

biggest_profit 106.25
daily_drawdown -180.25
max_drawown -962.0
avg_win 7.86491935483871
avg_loss -8.387777777777778
count_profit_trades 248
count_loss_trades 225
Win Rate1: 52.43%
win_rate 1.1
rrr 0.94


,order_type,profit
0,buy,-141.25
1,sell,204.50


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,current_max,drawdown,dayofweek,month,year
0,closed,EURUSD,buy,25000,2025-03-20 12:24:00,1.08503,2025-03-21 00:00:00,1.08494,0,0,...,-2.25,-1.75,-4.0,-4.0,9996.0,-4.00,0.0,4,2025-03,2025
1,closed,EURUSD,buy,25000,2025-03-21 18:53:00,1.08077,2025-03-24 00:00:00,1.08127,0,0,...,12.50,-1.75,10.75,6.75,10006.75,6.75,0.0,0,2025-03,2025
2,closed,EURUSD,sell,25000,2025-03-27 18:31:00,1.08007,2025-03-28 00:00:00,1.08011,0,0,...,-1.00,-1.75,-2.75,4.0,10004.0,6.75,-2.75,4,2025-03,2025
3,closed,EURUSD,sell,25000,2025-03-28 18:23:00,1.08378,2025-03-31 00:00:00,1.08225,0,0,...,38.25,-1.75,36.5,40.5,10040.5,40.50,0.0,0,2025-03,2025
4,closed,EURUSD,sell,25000,2025-04-02 17:41:00,1.08282,2025-04-02 17:52:00,1.08523,0,0,...,-60.25,-1.75,-62.0,-21.5,9978.5,40.50,-62.0,2,2025-04,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,closed,EURUSD,buy,25000,2025-04-30 23:52:00,1.13229,2025-04-30 23:53:00,1.13233,0,0,...,1.00,-1.75,-0.75,-878.5,9121.5,82.50,-961.0,2,2025-04,2025
480,closed,EURUSD,buy,25000,2025-04-30 23:54:00,1.13241,2025-04-30 23:55:00,1.13244,0,0,...,0.75,-1.75,-1.0,-879.5,9120.5,82.50,-962.0,2,2025-04,2025
481,closed,EURUSD,buy,25000,2025-04-30 23:56:00,1.13249,2025-04-30 23:57:00,1.13267,0,0,...,4.50,-1.75,2.75,-876.75,9123.25,82.50,-959.25,2,2025-04,2025
482,closed,EURUSD,buy,25000,2025-04-30 23:58:00,1.13272,2025-05-01 00:00:00,1.13279,0,0,...,1.75,-1.75,0.0,-876.75,9123.25,82.50,-959.25,3,2025-05,2025


ValueError: Plotly Express cannot process wide-form data with columns of different type.

In [ ]:
# visualizing the ohlc data
fig = go.Figure(data = [go.Candlestick(x = ohlc_df['time'],
                                        open = ohlc_df['open'],
                                        high = ohlc_df['high'],
                                        low = ohlc_df['low'],
                                        close = ohlc_df['close'])])

# fig.update_layout(height=600, xaxis_rangeslider_visible=False)
# 📌 Add Pivot Points as Horizontal Lines
pivot_levels = ["sell_stope_loss", "R2", "buy_stope_loss", "S2"]
colors = {'sell_stope_loss': 'red', 'buy_stope_loss': 'red', 'R2': 'green', 'S2': 'green'}


for _, row in daily_df.iterrows():
    for level in pivot_levels:
        fig.add_trace(go.Scatter(
            x=[row['time'], row['time'] + pd.Timedelta(days=1)],  # Extend for one day
            y=[row[level], row[level]],
            mode='lines',
            name=f"{level} {row['time']}",
            line=dict(color=colors[level], width=1.5, dash='dot')
        ))

# 📌 Customize Layout
fig.update_layout(
    title="Pivot Points on Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_dark",
    xaxis_rangeslider_visible=False
)
fig